In [1]:
import sys
sys.path.append('../src/')
import matplotlib.pyplot as plt
from utils.visual_functions import *
from utils.utils import *
from net.model import ReccurrentBlock
import numpy as np
import torch 
from sklearn.manifold import TSNE
import matplotlib
#import seaborn as sns
#sns.color_palette('husl', n_colors=20)
%matplotlib inline
fig_path="../figure/paper/"
seed = 4783957
print("set seed")
np.random.seed(seed)
plt.style.use(['science','ieee', 'muted'])
import torch
import torch.nn as nn
import torch.nn.functional as F

set seed


In [2]:
def load_data(data_type="plaid", submetered=True):
    
    path_sub = f"../data/{data_type}/submetered/" if submetered else f"../data/{data_type}/aggregated/"
    
    current = np.load(path_sub+"current.npy")
    voltage = np.load(path_sub+"voltage.npy")
    labels = np.load(path_sub+"labels.npy")
    return current, voltage, labels

def calculatePower(train_current, train_voltage, NN=500):
  

    n = len(current)
    PQ = np.empty([n,2])
    for i in range(n):
        # extract current and voltage in two cycle
        # extract current and voltage in two cycle
        temp_I = current[i]
        temp_V = voltage[i]
        Irms = np.mean(temp_I**2)**0.5
        Vrms = np.mean(temp_V**2)**0.5

        # Instantaneous power: obtained by multiplying the instantaneous voltage and current values
        Pinst = temp_I * temp_V
        Pactive = np.mean(Pinst) 

        # Apparent power: obtained by multiplying the RMS values of voltage and current
        Papparent = Vrms*Irms;

        # Ractive power if power triangle holds:
        Preactive = np.sqrt(Papparent**2 - Pactive**2);




        PQ[i,0]=Pactive
        PQ[i,1]=Preactive
    return PQ



In [3]:
def plot_PQPlane(labels, PQ, legends=None, sample=True):
    
    if legends:
        Legends=legends
    else:
        Legends=np.unique(labels)
        
    clrs = sns.color_palette("Paired", len(Legends))
    markers='1v^<P4ph+DX*8os3<-'
    
    minx,maxx,miny,maxy = (0,0,0,0)
    

    for i, app in enumerate(Legends):
        ix=np.where(labels==app)[0]
        if sample:
            ix1 = np.random.randint(PQ.shape[0],size=6)
            tempx = PQ[ix1,0]
            tempy = PQ[ix1,1]
        else:
            tempx = PQ[ix,0]
            tempy = PQ[ix,1]
        plt.scatter(tempx,tempy,marker=markers[i], s=30, label=Legends[i], color=clrs[i])
        minx = min(minx,min(tempx))
        maxx = max(maxx,max(tempx))
        miny = min(miny,min(tempy))
        maxy = max(maxy,max(tempy))    
    plt.axis([minx*0.9,maxx*1.1,miny*0.9,maxy*1.1])
    plt.xlabel('Real Power(W)')
    plt.ylabel('Reactive Power (VAR)')
    plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

    
def tsne_visualizer(labels, feature, legends=None):
    if legends:
        Legends=legends
    else:
        Legends=np.unique(labels)
    clrs = sns.color_palette("Paired", len(Legends))
    model  = TSNE(perplexity=40, min_grad_norm=1E-12, n_iter=3000)
    #model = TSNE(n_components=2, random_state=0, perplexity=20, n_iter=1000)
    TSNEF  = model.fit_transform(feature.reshape(len(feature), -1))
    
    for i, app in enumerate(Legends):
        ix=np.where(labels==app)[0]
        plt.scatter(TSNEF[ix,0], TSNEF[ix,1],label=app, color=clrs[i])
    plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
    plt.xlabel("tsne-2d-one")
    plt.ylabel("tsne-2d-two")

### PLAID

In [4]:
current, voltage, labels = load_data("plaid", submetered=False)
plaid_label =[]
for l in labels:
    idx=l-1 if l<4 else l-2
    plaid_label.append(list(plaid_labels.values())[idx])

## Label distributions

In [5]:
label, counts=np.unique(plaid_label, return_counts=True)
volgorde = np.argsort(counts)[::-1][:len(counts)]
fig = figure(columns=2)
plt.bar(label[volgorde], counts[volgorde], color=colors[0])
plt.xticks(rotation=90, fontsize=16)
ax = plt.gca()  
ax.set_ylabel(f'Number of activations', fontsize=12)
ax.set_xlabel("", fontsize=16)
ax.tick_params(axis='both', which='major')  
ax.autoscale(tight=True)
ax = plt.gca()
ax.tick_params(axis="both", which="minor", bottom=False, 
               top=False, labelbottom=True, left=False, right=False, labelleft=True)
format_axes(ax)
savefig(fig_path+"label_distribution_plaid",format=".pdf")

In [6]:
fig = figure(fig_width=None, fig_height=None, columns=1)
plt.plot(current[0], "k")
plt.ylabel("$i(t)$ $(A)$")
savefig(fig_path+"activation_current", format=".png")

In [9]:
fig = figure(fig_width=None, fig_height=None, columns=1)
plt.plot(current[0], "k")
plt.ylabel("$i(t)$ $(A)$")
savefig(fig_path+"activation_current", format=".pdf")
fig = figure(fig_width=None, fig_height=None, columns=1)
plt.plot(voltage[0], "k")
plt.ylabel("$v(t)$ $(V)$")
savefig(fig_path+"activation_voltage", format=".pdf")

In [ ]:
from IPython import display
for idx in np.where(np.array(plaid_label)=="CFL")[0]:
    plt.title(idx)
    plt.plot(current[idx], "k--", label="$i(t)$", alpha=0.8)
    plt.show()
    input("enter")
    display.clear_output()

In [12]:
fig = figure(fig_width=None, fig_height=None, columns=2)
plt.plot(current[11], "k")
plt.ylabel("$i(t)$ $(A)$")
savefig(fig_path+"activation_current_noise_cfl", format=".pdf")

### PLAID voltages

In [ ]:
with plt.style.context(['science', 'ieee']):
    fig, axs = plt.subplots(len(label)//3,3, sharex=True, figsize=(6, 4))
    fig.subplots_adjust(hspace = 0.25)
    axs = axs.ravel()
    for idap, app in enumerate(label):
        idx = np.random.choice(np.where(np.array(plaid_label)==app)[0])
        #i_a, i_f = fryze_power_decomposition(current_sample[idx], voltage_sample[idx], T=500)
        axs[idap].plot(voltage[idx], "k--", label="$v(t)$", alpha=0.8)
        #axs[idap].plot(i_a, label="$i(t)_a$")
        #axs[idap].plot(i_f, label="$i(t)_f$")
        axs[idap].set_title(app)
        axs[idap].set_ylabel("Voltage $(V)$") 
        axs[idap].set_xlim(0, 501) 
        format_axes(axs[idap])
        axs[idap].autoscale(tight=True)
    handles, labels = axs[idap].get_legend_handles_labels()
    #fig.legend(handles, labels, loc='upper center',  ncol=3, fontsize=14)
    fig.tight_layout()
    savefig(fig_path+"plaid_voltage",format=".pdf")

### Current PLAID

In [ ]:
with plt.style.context(['science', 'ieee']):
    fig, axs = plt.subplots(len(label)//3,3, sharex=True, figsize=(6, 4))
    fig.subplots_adjust(hspace = 0.25)
    axs = axs.ravel()
    for idap, app in enumerate(label):
        idx = np.random.choice(np.where(np.array(plaid_label)==app)[0])
        #i_a, i_f = fryze_power_decomposition(current_sample[idx], voltage_sample[idx], T=500)
        axs[idap].plot(current[idx], "k--", label="$i(t)$", alpha=0.8)
        #axs[idap].plot(i_a, label="$i(t)_a$")
        #axs[idap].plot(i_f, label="$i(t)_f$")
        axs[idap].set_title(app)
        axs[idap].set_ylabel("Current $(A)$") 
        axs[idap].set_xlim(0, 501) 
        format_axes(axs[idap])
        axs[idap].autoscale(tight=True)
    handles, labels = axs[idap].get_legend_handles_labels()
    #fig.legend(handles, labels, loc='upper center',  ncol=3, fontsize=14)
    fig.tight_layout()
    savefig(fig_path+"plaid_current",format=".pdf")

### VI

In [ ]:
with plt.style.context(['science', 'ieee']):
    fig, axs = plt.subplots(len(label)//3,3, sharex=True, figsize=(6, 4))
    fig.subplots_adjust(hspace = 0.25)

    axs = axs.ravel()
    for idap, app in enumerate(label):
        idx = np.random.choice(np.where(np.array(plaid_label)==app)[0])
        #i_a, i_f = fryze_power_decomposition(current_sample[idx], voltage_sample[idx], T=500)
        axs[idap].plot(voltage[idx], current[idx], "k--", label="$i(t)$", alpha=0.8)
        #axs[idap].plot(i_a, label="$i(t)_a$")
        #axs[idap].plot(i_f, label="$i(t)_f$")
        axs[idap].set_title(app)
        axs[idap].set_ylabel("Current $(A)$") 
        axs[idap].set_xlabel("Voltage $(V)$") 

        format_axes(axs[idap])
        axs[idap].autoscale(tight=True)
    handles, labels = axs[idap].get_legend_handles_labels()
    #fig.legend(handles, labels, loc='upper center',  ncol=3, fontsize=14)
    fig.tight_layout()
    savefig(fig_path+"plaid_vi",format=".pdf")

In [10]:
plaid_label = np.array(plaid_label)
cfl = np.where(plaid_label=="CFL")[0]

In [35]:
fig = figure(fig_height=1.8, fig_width=2.0)
x = current[cfl[0]]
xpaa = paa(x, 50)
y = voltage[cfl[0]]
ypaa = paa(y, 50)
plt.plot(x, y, "k")
plt.ylabel("voltage $(V)$", fontsize=12)
plt.xlabel("current $(A)$",  fontsize=12)
plt.tight_layout()
savefig(fig_path+"VI_plaid",format=".pdf")

In [31]:
fig = figure(fig_height=2.2, fig_width=3)
x = current[cfl[0]]
xpaa = paa(x, 50)
y = voltage[cfl[0]]
ypaa = paa(y, 50)
plt.plot(y, "k")
plt.ylabel("voltage $(V)$", fontsize=12)
plt.xlabel("sample $(T_s)$",  fontsize=12)
plt.tight_layout()
savefig(fig_path+"voltage_plaid",format=".pdf")

In [25]:
fig = figure(fig_height=2.2, fig_width=3)
x = current[cfl[0]]
xpaa = paa(x, 50)
y = voltage[cfl[0]]
ypaa = paa(y, 50)
plt.plot(x, "k")
plt.ylabel("current $(A)$", fontsize=12)
plt.xlabel("sample $(T_s)$",  fontsize=12)
plt.tight_layout()
savefig(fig_path+"current_plaid",format=".pdf")

In [29]:
img = get_img_from_VI(x, y, 50,1,para=1)
fig = figure(fig_height=1.5, fig_width=2.0)
plt.imshow(img, cmap='binary')
plt.xlabel("$w$")
plt.ylabel("$w$")
plt.tight_layout()
savefig(fig_path+"VI_image_plaid",format=".pdf")

In [ ]:
plt.plot(xpaa)
plt.xlim(0, 50)
plt.xlabel("sample $w$")
plt.ylabel("current $(A)$")
plt.tight_layout()
#savefig(fig_path+"after_PAA",format=".pdf")

In [36]:
RP = generate_input_feature(current, voltage, "adaptive", width=50, multi_dimension=True).numpy()

processed: 1478: : 1478it [00:14, 101.32it/s]


In [49]:
eps=10
delta=10
dist = torch.tensor(RP[cfl[0]][0])
dist = torch.floor(dist*eps)
dist[dist>delta]=delta
fig = figure(fig_height=1.5, fig_width=2.0)
plt.imshow(dist, cmap='binary', origin='lower')
plt.xlabel("$w$")
plt.ylabel("$w$")
plt.tight_layout()
savefig(fig_path+"WRG_image_plaid",format=".pdf")

In [43]:
fig = figure(fig_height=1.5, fig_width=2.0)
plt.imshow(RP[cfl[0]][0], cmap='gray', origin='lower')
plt.xlabel("$w$")
plt.ylabel("$w$")
plt.tight_layout()
savefig(fig_path+"WRG_image_plaid",format=".pdf")

In [ ]:
PQ=calculatePower(current, voltage, NN=500)

In [ ]:
fig=figure(fig_width=7, fig_height=5)
plot_PQPlane(plaid_label, PQ, None, sample=False)
savefig(fig_path+"plaid_PQ_plane",format=".pdf")

In [ ]:
fig =figure(fig_width=3, fig_height=3)
dist = create_distance_similarity_matrix(x, 50, 1)
model = ReccurrentBlock()
out = model(dist)
img = out.detach()[0]
plt.imshow(img, cmap='YlGnBu', origin='lower')
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"rcp_plaid",format=".pdf")

In [ ]:
fig = figure(fig_width=3, fig_height=3)
c = current[cfl[0]]
v = voltage[cfl[0]]
img=get_img_from_VI(c,v, 50,True,1)
plt.imshow(img, cmap='binary')
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"plaid_vi_image",format=".pdf")

### Feature embending 

In [ ]:
RP = generate_input_feature(current, voltage, "adaptive", width=50, multi_dimension=True).numpy()
VI = generate_input_feature(current, voltage, "vi", width=50, multi_dimension=True).numpy()

In [ ]:
fig=figure(fig_width=6, fig_height=6)
tsne_visualizer(plaid_label, VI, legends=None)
savefig(fig_path+"tsne_feature_VI",format=".pdf")

In [ ]:
fig=figure(fig_width=6, fig_height=6)
tsne_visualizer(plaid_label, RP, legends=None)
savefig(fig_path+"tsne_feature_RP",format=".pdf")

In [ ]:
fig=figure(fig_width=6, fig_height=6)
tsne_visualizer(plaid_label, current, legends=None)
savefig(fig_path+"tsne_feature_current",format=".pdf")

In [ ]:
fig=figure(fig_width=6, fig_height=6)
tsne_visualizer(plaid_label, voltage, legends=None)
savefig(fig_path+"tsne_feature_voltage",format=".pdf")

## Visualize learned representation

In [ ]:
class Conv2DVI(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.2):
        super(Conv2DVI, self).__init__()
        self.in_size=in_size
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_size, 16, 5, 2),
            nn.ReLU()
            #nn.MaxPool2d(3, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 2),
            nn.ReLU()
            #nn.MaxPool2d(3, 2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 5, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.fc_out=nn.Sequential(
            nn.Linear(64, 1024),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(1024, out_size)
            )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x= self.fc_out(x)
        return x

In [ ]:
from data_generator import *
from model import *
from utils import *
from fit_functions import *
dataset="plaid"
image_type="vi"
width=50
model_name="CNN"
run_id=1
file_name=f"{dataset}_{image_type}_{str(width)}_{model_name}_{str(run_id)}__exp_two"
saved_model_path   = '../checkpoint/plaid_vi_50_CNN_1__exp_two_checkpoint.pt'
checkpoint = Checkpoint(saved_model_path, patience=100, checkpoint=True, score_mode="max",min_delta=1e-4)
model_vi = Conv2DVI(1, out_size=12, dropout=0.2)
model_vi=checkpoint.load_saved_model(model_vi)
model_vi = model_vi.eval()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(labels)
y = le.transform(labels)
vi_data = Dataset(torch.tensor(VI), y)
loader=torch.utils.data.DataLoader(vi_data, 32, shuffle=False, num_workers=4, drop_last=False)
target = []
latent_feature = []
with torch.no_grad():
    for i, data in enumerate(loader):
        inp, targ=data
        out = model_vi(inp)
        latent_feature.append(out.data.numpy())
        target.append(targ.numpy())

In [ ]:
target = np.hstack(target)
latent_feature=np.concatenate(latent_feature, 0)
model = TSNE(perplexity=40, min_grad_norm=1E-12, n_iter=3000)
TSNEF  = model.fit_transform(latent_feature)

In [ ]:
fig=figure(fig_width=6, fig_height=6)
clrs =sns.color_palette("Paired", 12)
for i in np.unique(target):
    ix=np.where(target==i)[0]
    plt.scatter(TSNEF[ix,0], TSNEF[ix,1],label=plaid_names[i], color=clrs[i])
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
plt.xlabel("tsne-2d-one")
plt.ylabel("tsne-2d-two")
savefig(fig_path+"tsne_latent_feature_VI_plaid",format=".pdf")

### RP

In [ ]:
image_type="adaptive"
width=50
model_name="CNN"
run_id=1
saved_model_path   = '../checkpoint/plaid_adaptive_50_CNN_1_parameters_emb_size_10_10_checkpoint.pt'
checkpoint = Checkpoint(saved_model_path, patience=100, checkpoint=True, score_mode="max",min_delta=1e-4)
model_rp = Conv2DAdaptiveRecurrence(in_size=1, out_size=12,
                                            dropout=0.2)
model_rp=checkpoint.load_saved_model(model_rp)
model_rp=model_rp
rp_data = Dataset(torch.tensor(RP), y)
loader=torch.utils.data.DataLoader(rp_data, 32, shuffle=False, num_workers=4, drop_last=False)
target = []
latent_feature = []
with torch.no_grad():
    for i, data in enumerate(loader):
        inp, targ=data
        out = model_rp(inp)
        latent_feature.append(out.data.numpy())
        target.append(targ.numpy())

In [ ]:
target = np.hstack(target)
latent_feature=np.concatenate(latent_feature, 0)
model = TSNE(perplexity=40, min_grad_norm=1E-12, n_iter=3000)
TSNEF  = model.fit_transform(latent_feature)

In [ ]:
fig=figure(fig_width=6, fig_height=6)
clrs =sns.color_palette("Paired", 12)
for i in np.unique(target):
    ix=np.where(target==i)[0]
    plt.scatter(TSNEF[ix,0], TSNEF[ix,1],label=plaid_names[i], color=clrs[i])
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
plt.xlabel("tsne-2d-one")
plt.ylabel("tsne-2d-two")
savefig(fig_path+"tsne_latent_feature_RP_plaid",format=".pdf")

### Industrial

In [13]:
current, voltage, labels = load_data("lilac", submetered=False)

In [16]:
lilacd_label = []
for l in labels:
    lilacd_label.append(lilac_labels[l])

In [28]:
label, counts=np.unique(lilacd_label, return_counts=True)
volgorde = np.argsort(counts)[::-1][:len(counts)]
fig = figure(columns=2)
plt.bar(label[volgorde], counts[volgorde], alpha=0.6)
plt.xticks(rotation=90, fontsize=16)
ax = plt.gca()  
ax.set_ylabel(f'Number of activations', fontsize=16)
ax.set_xlabel("", fontsize=20)
ax.tick_params(axis='both', which='major')  
ax.autoscale(tight=True)
ax = plt.gca()
ax.tick_params(axis="both", which="minor", bottom=False, 
               top=False, labelbottom=True, left=False, right=False, labelleft=True)
format_axes(ax)
savefig(fig_path+"label_distribution_lilac",format=".pdf")

In [ ]:
np.unique(labels)
res_app=['Coffee-machine', 'Drilling-machine', 'Kettle', 'Raclette', 'Refrigerator', 'Vacuum-cleaner', 'Hair-dryer', 'Bulb']
indust_app = []
for l in np.unique(labels):
    if l not in res_app:
        indust_app.append(l)

In [ ]:
## three phase
motor = np.where(labels=='Squirrel-3-async-2x')[0]
x = current[motor[20]]

In [ ]:
fig =figure(fig_width=5, fig_height=4)
plt.plot(x[0], label="phase-1")
plt.xlabel("sample $N$")
plt.ylabel("current $(A)$")
plt.xlim(0, 1000)
plt.tight_layout()
savefig(fig_path+"input_signal_industrial_three_phase_1",format=".pdf")

In [ ]:
fig =figure(fig_width=5, fig_height=4)
plt.plot(x[1], label="phase-2")
plt.xlabel("sample $N$")
plt.ylabel("current $(A)$")
plt.xlim(0, 1000)
plt.tight_layout()
savefig(fig_path+"input_signal_industrial_three_phase_2",format=".pdf")

In [ ]:
fig =figure(fig_width=5, fig_height=4)
plt.plot(x[2], label="phase-3")
plt.xlabel("sample $N$")
plt.ylabel("current $(A)$")
plt.xlim(0, 1000)
plt.tight_layout()
savefig(fig_path+"input_signal_industrial_three_phase_3",format=".pdf")

In [ ]:
fig =figure(fig_width=4, fig_height=3)
dril = np.where(labels=='Drilling-machine')[0]
y = current[dril[10]]
v = voltage[dril[10]]
plt.plot(y[0], label="phase-1")
plt.plot(y[1], label="phase-2")
plt.plot(y[2], label="phase-3")
plt.legend()
plt.xlabel("sample $N$")
plt.ylabel("current $A$")
#plt.ylim(-1,1)
plt.tight_layout()
savefig(fig_path+"input_signal_industrial_single_phase_DL",format=".pdf")

In [ ]:
## PQ plane

In [ ]:
PQ = []
for k in range(3):
    PQ+=[calculatePower(current[:,k,:], voltage[:,k,:], NN=500)]
PQ = np.array(PQ)

In [ ]:
PQT=PQ.sum(0)
fig=figure(fig_width=8)
plot_PQPlane(labels, PQT, None, sample=False)
savefig(fig_path+"lilac_PQ_plane",format=".pdf")

In [ ]:
dist = create_N_distance_similarity_matrix(x, 50, 1)
model = ReccurrentBlock()
out = model(dist)
img = out.detach()[0]
fig = figure(fig_width=3, fig_height=3)
plt.imshow(img, cmap='YlGnBu', origin='lower')
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"rcp_lilac_one_sq_1",format=".pdf")

In [ ]:

out = model(dist)
img = out.detach()[1]
fig = figure(fig_width=3, fig_height=3)
plt.imshow(img, cmap='YlGnBu', origin='lower')
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"rcp_lilac_one_sq_2",format=".pdf")

In [ ]:

img = out.detach()[2]
fig = figure(fig_width=3, fig_height=3)
plt.imshow(img, cmap='YlGnBu', origin='lower')
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"rcp_lilac_one_sq_3",format=".pdf")

In [ ]:
dist = create_N_distance_similarity_matrix(y, 50, 1)
model = ReccurrentBlock()
out = model(dist)
img = out.detach()[0]
fig = figure(fig_width=3, fig_height=3)
plt.imshow(img, cmap='YlGnBu', origin='lower')
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"rcp_lilac_phase_drill_1",format=".pdf")

In [ ]:
img = out.detach()[1]
fig = figure(fig_width=3, fig_height=3)
plt.imshow(img, cmap='YlGnBu', origin='lower')
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"rcp_lilac_phase_2_drill",format=".pdf")

In [ ]:
img = out.detach()[2]
fig = figure(fig_width=3, fig_height=3)
plt.imshow(img, cmap='YlGnBu', origin='lower')
plt.xlabel("$W$")
plt.ylabel("$H$")
plt.tight_layout()
savefig(fig_path+"rcp_lilac_phase_3_drill",format=".pdf")

In [ ]:
RP = generate_input_feature(current, voltage, "adaptive", width=50, multi_dimension=True).numpy()
#

In [ ]:
VI = generate_input_feature(current, voltage, "vi", width=50, multi_dimension=True).numpy()

In [ ]:
fig=figure(fig_width=6, fig_height=6)
tsne_visualizer(labels, VI, legends=None)
savefig(fig_path+"tsne_feature_VI_lilac",format=".pdf")

In [ ]:
fig=figure(fig_width=6, fig_height=6)
tsne_visualizer(labels, RP, legends=None)
savefig(fig_path+"tsne_feature_RP_lilac",format=".pdf")

## Visualize learned representation VI

In [ ]:
dataset="lilac"
image_type="vi"
width=50
model_name="CNN"
run_id=1
saved_model_path   = '../checkpoint/lilac_vi_50_CNN_1__exp_two_checkpoint.pt'
checkpoint = Checkpoint(saved_model_path, patience=100, checkpoint=True, score_mode="max",min_delta=1e-4)
model = Conv2DVI(3, out_size=16, dropout=0.2)
model_vi=checkpoint.load_saved_model(model)
model_vi = model_vi.eval()


In [ ]:
le = LabelEncoder()
le.fit(labels)
y = le.transform(labels)
vi_data = Dataset(torch.tensor(VI), y)
loader=torch.utils.data.DataLoader(vi_data, 32, shuffle=False, num_workers=4, drop_last=False)
target = []
latent_feature = []
with torch.no_grad():
    for i, data in enumerate(loader):
        inp, targ=data
        out = model_vi(inp)
        latent_feature.append(out.data.numpy())
        target.append(targ.numpy())

In [ ]:
target = np.hstack(target)
latent_feature=np.concatenate(latent_feature, 0)
model = TSNE(perplexity=40, min_grad_norm=1E-12, n_iter=3000)
TSNEF  = model.fit_transform(latent_feature)

In [ ]:
fig=figure(fig_width=6, fig_height=6)
clrs =sns.color_palette("Paired", 16)
for i in np.unique(target):
    ix=np.where(target==i)[0]
    plt.scatter(TSNEF[ix,0], TSNEF[ix,1],label=lilac_names[i], color=clrs[i])
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
plt.xlabel("tsne-2d-one")
plt.ylabel("tsne-2d-two")
savefig(fig_path+"tsne_latent_feature_VI_lilac",format=".pdf")

In [ ]:
### RP

In [ ]:
image_type="adaptive"
width=50
model_name="CNN"
run_id=1
file_name=f"{dataset}_{image_type}_{str(width)}_{model_name}_{str(run_id)}__exp_two"
saved_model_path   = '../checkpoint/lilac_adaptive_50_CNN_1_parameters_emb_size_10_10_checkpoint.pt'
checkpoint = Checkpoint(saved_model_path, patience=100, checkpoint=True, score_mode="max",min_delta=1e-4)
model_rp = Conv2DAdaptiveRecurrence(in_size=3, out_size=16,
                                            dropout=0.2)
model_rp=checkpoint.load_saved_model(model_rp)
model_rp=model_rp
rp_data = Dataset(torch.tensor(RP), y)
loader=torch.utils.data.DataLoader(rp_data, 32, shuffle=False, num_workers=4, drop_last=False)
target = []
latent_feature = []
with torch.no_grad():
    for i, data in enumerate(loader):
        inp, targ=data
        out = model_rp(inp)
        latent_feature.append(out.data.numpy())
        target.append(targ.numpy())

In [ ]:
target = np.hstack(target)
latent_feature=np.concatenate(latent_feature, 0)
model = TSNE(perplexity=40, min_grad_norm=1E-12, n_iter=3000)
TSNEF  = model.fit_transform(latent_feature)

In [ ]:
fig=figure(fig_width=6, fig_height=6)
clrs =sns.color_palette("Paired", 16)
for i in np.unique(target):
    ix=np.where(target==i)[0]
    plt.scatter(TSNEF[ix,0], TSNEF[ix,1],label=lilac_names[i], color=clrs[i])
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
plt.xlabel("tsne-2d-one")
plt.ylabel("tsne-2d-two")
savefig(fig_path+"tsne_latent_feature_RP_lilac",format=".pdf")

### PLAID